In [1]:
import pandas as pd


df_wdc_co2 = pd.read_excel('data_for_plots.xlsx', sheet_name='WDC CO2', skiprows=1)
df_ngrip_d18o = pd.read_excel('data_for_plots.xlsx', sheet_name='NGRIP d18O', skiprows=1)
df_wdc_d18o = pd.read_excel('data_for_plots.xlsx', sheet_name='WDC d18O', skiprows=3)
df_do= pd.read_excel('data_for_plots.xlsx', sheet_name='DO', skiprows=1)
df_monsoon=pd.read_excel('data_for_plots.xlsx', sheet_name='monsoon', skiprows=1)
df_sealevel=pd.read_excel('data_for_plots.xlsx', sheet_name='sea level', skiprows=1)
df_ATS=pd.read_excel('data_for_plots.xlsx', sheet_name='ATS', skiprows=1)
df_composite_co2=pd.read_excel('data_for_plots.xlsx', sheet_name='composite CO2', skiprows=1)
df_LR04=pd.read_excel('data_for_plots.xlsx', sheet_name='LR04', skiprows=1)

# remove nan values for all three dataframes
df_wdc_co2 = df_wdc_co2.dropna()
df_ngrip_d18o = df_ngrip_d18o.dropna()
df_wdc_d18o = df_wdc_d18o.dropna()
df_monsoon=df_monsoon.dropna()
df_sealevel=df_sealevel.dropna()
df_ATS=df_ATS.dropna()
df_composite_co2=df_composite_co2.dropna()
df_LR04=df_LR04.dropna()


# rename columns to Age,Value
df_wdc_co2.columns = ['Age', 'Value', 'Error']
df_ngrip_d18o.columns = ['Age', 'Value']
df_wdc_d18o.columns = ['Age', 'Value']
df_monsoon.columns=['Age', 'Value']
df_sealevel.columns = ['Age', 'Value', 'Error']
df_ATS.columns=['Age', 'Value']
df_composite_co2.columns=['Age', 'Value', 'Error']
df_LR04.columns=['Age', 'Value','Error']


# drop values in the second column of df_wdc_d18o that above 999
df_wdc_d18o = df_wdc_d18o[df_wdc_d18o.iloc[:, 1] < 999]

# Age *-1
df_wdc_co2.Age = df_wdc_co2.Age * -1
df_ngrip_d18o.Age = df_ngrip_d18o.Age * -1
df_wdc_d18o.Age = df_wdc_d18o.Age * -1
df_monsoon.Age=df_monsoon.Age*-1000
df_sealevel.Age=df_sealevel.Age*-1000
df_ATS.Age=df_ATS.Age*-1
df_composite_co2.Age=df_composite_co2.Age*-1
df_LR04.Age=df_LR04.Age*-1000


# read the LGMR data
import xarray as xr
ds_LGMR_mean = xr.open_dataset('LGMR_GMST_climo.nc')
# ds_LGMR_mean.info
df_lgmr = ds_LGMR_mean.to_dataframe().reset_index()
# df_lgmr.head()  

# rename columns 
df_lgmr.columns = ['Age', 'Value', 'Error']
# *1
df_lgmr.Age = df_lgmr.Age * -1
df_lgmr

c:\Users\zps19\.conda\envs\pz-env\lib\site-packages\xarray\coding\variables.py:148: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  condition |= data == fv


,Age,Value,Error
0,-100.0,13.488699,0.219001
1,-300.0,13.379682,0.192081
2,-500.0,13.367580,0.185994
3,-700.0,13.417512,0.169142
4,-900.0,13.521124,0.178768
...,...,...,...
115,-23100.0,7.095615,0.440728
116,-23300.0,7.171380,0.464787
117,-23500.0,7.177038,0.416002
118,-23700.0,7.250135,0.437566


In [2]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column, row
from bokeh.models import CheckboxButtonGroup, CustomJS, HoverTool, ColumnDataSource, Div, Range1d, RangeSlider, TextInput, Button
from bokeh.io import curdoc
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from bokeh.models import Whisker
from bokeh.models import Band
from bokeh.models import Div

def interpolate_and_smooth(df, resolution=10, smooth_width=100):
    interp_func = interp1d(df.iloc[:, 0], df.iloc[:, 1], kind='linear', bounds_error=False)
    interpolated_ages = np.arange(df.iloc[:, 0].min(), df.iloc[:, 0].max() + resolution, resolution)
    interpolated_values = interp_func(interpolated_ages)
    window_size = smooth_width // resolution
    smoothed_values = np.convolve(interpolated_values, np.ones(window_size)/window_size, mode='valid')
    adjusted_ages = interpolated_ages[len(interpolated_ages) - len(smoothed_values):]
    return adjusted_ages, smoothed_values

# Interpolating and smoothing for δ18O data
wdc_d18o_ages, wdc_d18o_smoothed = interpolate_and_smooth(df_wdc_d18o)
ngrip_d18o_ages, ngrip_d18o_smoothed = interpolate_and_smooth(df_ngrip_d18o)

# Create a dictionary of data series
data_series = {
    "LGMR": df_lgmr,
    "WDC CO2": df_wdc_co2,
    "WDC δ18O": df_wdc_d18o,
    "NGRIP δ18O": df_ngrip_d18o,
    'Speleothem δ18O':df_monsoon,
    'Sea level': df_sealevel,
    'Antarctic Temperature Stack':df_ATS,
    'Composite CO2':df_composite_co2,
    'LR04':df_LR04

}
button_labels=['1. LGMR','2. WDC CO₂', '3. WDC δ¹⁸O', '4. NGRIP δ¹⁸O','5. Speleothem δ¹⁸O', '6. Sea level','7. Antarctic Temperature Stack','8. Composite CO₂','9. LR04 δ¹⁸O']

wdc_d18o_ages=wdc_d18o_ages/1000
ngrip_d18o_ages=ngrip_d18o_ages/1000

# Create Bokeh plot for each series
plots = []
xlabels=['Age (kyr relative to AD 1950)',
         'WD2014 Age (kyr, relative to AD 1950)', 
         'WD2014 Age (kyr, relative to AD 1950)', 
         'GICC05 Age (kyr, relative to AD 2000)',
         'Age (kyr relative to AD 1950)',
         'Age (kyr relative to AD 1950)',
         'Age (kyr relative to AD 1950)',
         'Age (kyr relative to AD 1950)',
         'Age (kyr relative to AD 1950)'
         ]
# ylabels=['CO2 (ppm)', 'δ18O (per mil)', 'δ18O (per mil)']
ylabels=['Global Mean Surface Temperature (°C)','CO₂ (ppm)', 'δ¹⁸O (permil)', 'δ¹⁸O (permil)','δ¹⁸O (permil)','Sea level (m)','Temperature (°C)','CO₂ (ppm)','δ¹⁸O (permil)']

y_ranges=[(5.3,14),(180, 290), (-44, -30), (-47, -32),(-3.7,-12.5),(-150,30),(-10,5),(170, 300),(5.3,2.5)]


y_range_middles=0.5*(np.array(y_ranges)[:,0]+np.array(y_ranges)[:,1])
y_hight=(np.array(y_ranges)[:,1]-np.array(y_ranges)[:,0])
DO_strips_dict = {}
DO_labels_dict = {}



for i, (name, df) in enumerate(data_series.items()):
    
    # if "DO" in name:
    #     continue

    source = ColumnDataSource(data=dict(x=df.iloc[:, 0]/1000, y=df.iloc[:, 1]))
    p = figure(sizing_mode='stretch_both', title=button_labels[i])
    p.y_range = Range1d(*y_ranges[i])
    
    # Plot GS
    x = df_do['middle of GS'].values*-1/1000
    y = y_range_middles[i]*np.ones(len(x))
    width = df_do['GS duration'].values/1000
    height = y_hight[i]*np.ones(len(x))

    DO_strips=p.rect(x, 
            y, 
            width, 
            height,
            fill_color="#ebdbb2",
            line_color="#ebdbb2"
            )# "#cab2d6"
    DO_strips.level = 'underlay' 
    DO_strips.visible = False
    DO_strips_dict[name] = DO_strips

    # add text annotation on the middle of GS, with 90 degree rotation
    # p.text(x=x+width/2, y=y, text=df_do['GS'].values, text_align='center', text_baseline='middle', text_font_size='8pt', angle=np.pi/2)
    DO_labels=p.text(x=df_do['middle of GS'].values*-1/1000, 
            y=(y_range_middles[i]+0.34*y_hight[i])*np.ones(len(x)),
            text=df_do['event'].values,
            text_font_size='8pt',
            text_align='center',
            text_baseline='middle',
            text_color='black',
            angle=np.pi/2
            
            )
    # set the order to be just above the strips
    DO_labels.level = 'glyph'
    DO_labels.visible = False
    DO_labels_dict[name] = DO_labels
    
    # plot the original data as a line plot
    p.line('x', 'y', source=source, line_width=2)

    # Check if 'Error' column exists and add error bands
    if 'Error' in df.columns:
        band_source = ColumnDataSource(data={
            'base': df.iloc[:, 0]/1000, 
            'lower': df.iloc[:, 1] - df['Error'], 
            'upper': df.iloc[:, 1] + df['Error']
        })

        band = Band(base='base', lower='lower', upper='upper', source=band_source, level='underlay',
                    fill_alpha=0.6, line_width=0, fill_color='gray')
        p.add_layout(band)
        
    p.visible = False
    p.xaxis.axis_label = xlabels[i]
    p.yaxis.axis_label = ylabels[i]
    p.x_range = Range1d(-70, -10)
    hover = HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")])
    p.add_tools(hover)

    if "δ18O" in name:
        if "WDC" in name:
            smoothed_source = ColumnDataSource(data=dict(x=wdc_d18o_ages, y=wdc_d18o_smoothed))
            p.line('x', 'y', source=smoothed_source, line_width=2, color="red", legend_label="100 yr moving average")
            p.legend.location = "top_left"
        elif "NGRIP" in name:
            smoothed_source = ColumnDataSource(data=dict(x=ngrip_d18o_ages, y=ngrip_d18o_smoothed))
            p.line('x', 'y', source=smoothed_source, line_width=2, color="red", legend_label="100 yr moving average")
            p.legend.location = "top_left"

    plots.append(p)

# Set the CO2 plot to be visible initially
plots[0].visible = True
# plots[2].visible = True

# Create RangeSlider widget for x-axis range
x_range_slider = RangeSlider(start=-70, end=-10, value=(-70, -10), step=0.01, title="Age zoomer", direction='ltr', width=500)

# Create TextInput widgets for min and max data range
min_range_input = TextInput(value="-70", title="Age Range Oldest:")
max_range_input = TextInput(value="-10", title="Age Range Youngest:")

# Create a 'Set' Button
set_button = Button(label="Set", button_type="success", width=100, height=65)

# Callback for the 'Set' Button
set_button_callback = CustomJS(args=dict(min_range_input=min_range_input, max_range_input=max_range_input, x_range_slider=x_range_slider), code="""
    const min_range = parseFloat(min_range_input.value);
    const max_range = parseFloat(max_range_input.value);
    if (min_range < max_range) {
        x_range_slider.start = min_range;
        x_range_slider.end = max_range;
        x_range_slider.value = [min_range, max_range];
                               

        p.x_range.start = min_range;
        p.x_range.end = max_range;
                               
    } else {
        alert('Min range should be less than max range.');
    }
""")

set_button.js_on_click(set_button_callback)


set_button.js_on_click(set_button_callback)

# Callback for the RangeSlider
slider_callback = CustomJS(args=dict(plots=plots, x_range_slider=x_range_slider), code="""
    const start = x_range_slider.value[0];
    const end = x_range_slider.value[1];
    for (let p of plots) {
        p.x_range.start = start;
        p.x_range.end = end;
    }
""")
x_range_slider.js_on_change('value', slider_callback)


checkbox_buttons = CheckboxButtonGroup(labels=button_labels + ["10. Greenland Stadials"], active=[0], sizing_mode='scale_width')
checkbox_callback = CustomJS(args=dict(plots=plots, DO_strips_dict=DO_strips_dict, DO_labels_dict=DO_labels_dict, checkbox_buttons=checkbox_buttons), code="""
    for (let i = 0; i < plots.length; i++) {  // Exclude the last button which is for Greenland Stadials
        plots[i].visible = this.active.includes(i);
    }
    // Handle Greenland Stadials toggle separately
    let gsIndex = checkbox_buttons.labels.length-1;
    if (checkbox_buttons.active.includes(gsIndex)) {
        for (let key in DO_strips_dict) {
            DO_strips_dict[key].visible = true;
            DO_labels_dict[key].visible = true;
        }
    } else {
        for (let key in DO_strips_dict) {
            DO_strips_dict[key].visible = false;
            DO_labels_dict[key].visible = false;
        }
    }

""")

checkbox_buttons.js_on_change('active', checkbox_callback)



checkbox_buttons.js_on_change('active', checkbox_callback)

# Arrange layout
range_controls = row(min_range_input, max_range_input, set_button)


reference_text = Div(
    text="""<i>
    <br> Interactive Plots of Major Past Climate Records. Records are roughly sorted by their length and plotted on their original age scales. 
    For the WDC CO₂, Global sea level, the composite CO₂, And the LR04 data, the gray shaded areas indicate the ±1-sigma standard error.
    <br> <br> <br>
    <br> Data References:
    <br>1. 24 kyr Last Glacial Maximum Reanalysis (LGMR) Global Temperature Reconstructions, Osman et al., (2022), <a href="https://doi.org/10.1038/s41586-021-03984-4" target="_blank">DOI: 10.1038/s41586-021-03984-4</a>
    <br>2. 66 kyr WDC CO₂ data, Bauska et al., (2021), <a href="https://doi.org/10.1038/s41561-020-00680-2" target="_blank">DOI: 10.1038/s41561-020-00680-2</a>; 
    Marcott et al., (2014), <a href="https://doi.org/10.1038/nature13799" target="_blank">DOI: 10.1038/nature13799</a>
    <br>3. 66 kyr WDC δ¹⁸O data, WAIS, (2015), <a href="https://doi.org/10.1038/nature14401" target="_blank">DOI: 10.1038/nature14401</a>
    <br>4. 122 kyr NGRIP δ¹⁸O; Seierstad, et al. (2014), <a href="https://doi.org/10.1016/j.quascirev.2014.10.032" target="_blank">DOI: 10.1016/j.quascirev.2014.10.032</a>; 
    Rasmussen et al., (2014), <a href="https://doi.org/10.1016/j.quascirev.2014.09.007" target="_blank">DOI: 10.1016/j.quascirev.2014.09.007</a>
    <br>5. 640 kyr composite Chinese speleothem δ¹⁸O data, Cheng et al., (2016), <a href="https://doi.org/10.1038/nature18591" target="_blank">DOI: 10.1038/nature18591</a>
    <br>6. 798 kyr Global sea level data, Spratt & Lisiecki, <a href="https://doi.org/10.5194/cp-12-1079-2016" target="_blank">DOI: 10.5194/cp-12-1079-2016</a>
    <br>7. 800 kyr composite Antarctic temperature record, Parrenin et al., (2013) <a href="https://doi.org/10.1126/science.1226368" target="_blank">DOI: 10.1126/science.1226368</a>
    <br>8. 800 kyr composite CO₂ record, Bereiter et al., (2015), <a href="https://doi.org/10.1002/2014GL061957" target="_blank">DOI: 10.1002/2014GL061957</a>; Lüthi et al., (2008), <a href="https://doi.org/10.1038/nature06949" target="_blank">DOI: 10.1038/nature06949</a>
    <br>9. 5320 kyr LR04 benthic δ¹⁸O stack, Lisiecki & Raymo (2005), <a href="https://doi.org/10.1029/2004PA001071" target="_blank">DOI: 10.1029/2004PA001071</a>
    <br>10. Greenland Stadial timings are on the GICC05 timescale (relative to AD 2000), Rasmussen et al., (2014), <a href="https://doi.org/10.1016/j.quascirev.2014.09.007" target="_blank">DOI: 10.1016/j.quascirev.2014.09.007</a>
    <br> <br>Developed by P. Zheng<br> <br> <br>
    </i>""",
    width=1000, 
    height=80  # Increased height to accommodate multiple lines
)


# range_controls = row(min_range_input, max_range_input, set_button, align="end")
layout = column(checkbox_buttons, range_controls, x_range_slider, *plots, reference_text,sizing_mode='stretch_both')
# layout = column(checkbox_buttons, range_controls, x_range_slider, *plots, reference_text, js_div, sizing_mode='stretch_both')


curdoc().add_root(layout)

output_file("PastClimatePlots/index.html", title="PastClimatePlots")

show(layout)


